In [90]:
import jovian
import opendatasets as od
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
#import pyarrow.feather as feather
import os
import folium
import dask.dataframe as dd
%matplotlib inline

In [87]:
df=pd.read_csv('C:\\Users\\admin\\Downloads\\Parking.csv')

C:\Users\admin\AppData\Local\Temp\ipykernel_25184\3256347001.py:1: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('C:\\Users\\admin\\Downloads\\Parking.csv')


MemoryError: Unable to allocate 438. MiB for an array with shape (7, 8198207) and data type object

In [ ]:
violation_defs_and_fines = pd.read_csv('https://data.cityofnewyork.us/api/views/ncbg-6agr/rows.csv?accessType=DOWNLOAD')
violation_defs_and_fines

In [ ]:
df.info()

In [ ]:
cols_to_keep =['Registration State',
               'Plate Type',
               'Issue Date',
               'Violation Time',
               'Violation Location',
               'Violation Precinct', 
               'Issuer Precinct',
               'Issuer Code',
               'Violation Code',
               'Meter Number',
               'Violation County',
               'Issuer Squad',
               'Issuing Agency']

In [ ]:
ndf = df[cols_to_keep].copy() # making a copy detaches this from the origional sample_df and raw_df
ndf.head()

In [ ]:
precinct_loc_df = ndf[['Violation Location', 'Violation Precinct', 'Issuer Precinct']]
precinct_loc_df.describe()

In [ ]:
%%time
ndf['Issue Date and Time'] = ndf['Issue Date'] + ' ' + ndf['Violation Time']
#ndf['Issue Date and Time'] = ndf['Issue Date and Time'] + 'M'
#ndf['Issue Date and Time'] = pd.to_datetime(df['Issue Date and Time'], format='%m/%d/%Y %I%M%p', errors='coerce')

In [ ]:
ndf['Issue Date and Time'] = ndf['Issue Date and Time'] + 'M'

In [ ]:
ndf['Issue Date and Time'] = pd.to_datetime(ndf['Issue Date and Time'], format='%m/%d/%Y %I%M%p', errors='coerce')

In [ ]:
ndf['Issue Date and Time'].isna().sum()

In [ ]:
ndf['Issue Date and Time'].isna().sum()/ndf.shape[0] * 100

In [ ]:
ndf.drop(['Violation Time', 'Issue Date'], axis=1, inplace=True)
ndf.head(3)

In [ ]:
ndf[ndf['Violation County'].isin(['KINGS', 'QNS', 'BRONX'])]

In [ ]:
remap_county_dict = {
    'K' : 'Brooklyn',
    'Q' : 'Queens',
    'NY': 'Manhattan',
    'QN': 'Queens',
    'BK': 'Brooklyn',
    'R' : 'Staten Island',
    'BX': 'Bronx',
    'ST': 'Staten Island',
    'MN': 'Manhattan',
    'KINGS': 'Brooklyn',
    'QNS': 'Queens',
    'BRONX': 'Bronx'
}
ndf['Violation County'] = ndf['Violation County'].map(remap_county_dict).astype('category')
df.head(3)

In [ ]:
ndf['Violation County'].isna().sum()

In [ ]:
ndf['Violation County'].unique()

In [ ]:
county_totals_df = pd.DataFrame(ndf['Violation County'].value_counts())
county_totals_df

In [ ]:
counties = list(ndf['Violation County'][~ndf['Violation County'].isnull()].unique())
for county in counties:
    print('The number of precints in {} data are: {}'.format(county, 
                                              #np.sort(df['Violation Location'][df['Violation County'] == county].unique())))
                                              ndf['Violation Location'][ndf['Violation County'] == county].nunique()))

In [91]:
import json
from urllib.request import urlopen

precinct_geo = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'

with urlopen(precinct_geo) as response:
    precincts = json.load(response)

In [104]:
type(precincts)


dict

In [ ]:
pd.DataFrame([dict_])

In [78]:
with open("sample.json", "w") as outfile:
    json.dump(precincts, outfile)
    

In [92]:
feature_id_list = []
for index in range(len(precincts['features'])):
    feature_id_list.append(precincts['features'][index]['properties']['precinct'])
np.sort(feature_id_list)

array(['1', '10', '100', '101', '102', '103', '104', '105', '106', '107',
       '108', '109', '110', '111', '112', '113', '114', '115', '120',
       '121', '122', '123', '13', '14', '17', '18', '19', '20', '22',
       '23', '24', '25', '26', '28', '30', '32', '33', '34', '40', '41',
       '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '52',
       '6', '60', '61', '62', '63', '66', '67', '68', '69', '7', '70',
       '71', '72', '73', '75', '76', '77', '78', '79', '81', '83', '84',
       '88', '9', '90', '94'], dtype='<U3')

In [98]:
from pandas import json_normalize
df = json_normalize(precincts)

In [103]:
for row in range(len(precincts)):
    #First I load the dict (one at a time)
    n = precincts.loc[row,'dict_column']
    #Now I make a new column that pulls out the data that I want.
    precincts.loc[row,'new_column'] = n.get('key')

AttributeError: 'dict' object has no attribute 'loc'

In [101]:
recs = precincts['type']['features']
df = json_normalize(recs)
print(df.head())

TypeError: string indices must be integers, not 'str'

In [99]:
df.head()

,type,features
0,FeatureCollection,"[{'type': 'Feature', 'properties': {'precinct'..."


In [95]:
train = pd.DataFrame.from_dict(precincts, orient='index')
train.reset_index(level=0, inplace=True)

In [96]:
train.head()

,index,0
0,type,FeatureCollection
1,features,"[{'type': 'Feature', 'properties': {'precinct'..."


In [41]:
precinct_numb_df = ndf['Violation Location'][~(ndf['Violation Location'].isnull()) ]. \
                     astype('int32').value_counts().reset_index().rename(columns={'index': 'Violation Location',
                                                                                  'Violation Location': 'counts'})
# the values below need to match the string format of the GeoJson file
precinct_numb_df['Violation Location'] = precinct_numb_df['Violation Location'].astype('str')
# check the precict number is valid and in the geojso list above -- if it is not in the list exclude it
precinct_numb_df = precinct_numb_df[precinct_numb_df['Violation Location'].isin(feature_id_list)]
precinct_numb_df

,Violation Location,counts
0,19,209247
1,6,158075
2,114,157238
3,13,141205
4,14,138000
...,...,...
72,113,15455
73,101,12765
74,100,11494
75,123,6869


In [42]:
precinct_dict = {
    'Manhattan': list(range(1,35)),
    'Bronx': list(range(40,53)),
    'Brooklyn': list(range(60,95)),
    'Queens': list(range(100,116)),
    'Staten Island': list(range(120,124))
}

In [80]:
precinct_geo


'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'

In [82]:
feature.properties.precinct

NameError: name 'feature' is not defined

In [83]:
m = folium.Map(location=[40.730610, -73.935242], tiles='CartoDBpositron', zoom_start=10.5)

folium.Choropleth(
        geo_data=precinct_geo,                       # The geojson files from NYC open data
        name="choropleth",
        data=precinct_numb_df,                       # our grouped DF
        columns=["Violation Location", 'counts'],    # "Violation Location" aligns with feature.properties below
        key_on='feature.properties.precinct',        #  location that aligns with precinct
        fill_color="YlGnBu",
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name="Tickets issued 2017",
        print(key_on)
    ).add_to(m)

folium.LayerControl().add_to(m)
m

SyntaxError: positional argument follows keyword argument (2314724119.py, line 15)

In [75]:
precinct_numb_df.to_csv('C:\\Users\\admin\\Downloads\\precinct_numb_df_df.csv')

In [47]:
violation_defs_and_fines.columns = ['Violation Code', 'Violation Description', 'Fine (below 96th)', 'Fine']
violation_defs_and_fines.columns

Index(['Violation Code', 'Violation Description', 'Fine (below 96th)', 'Fine'], dtype='object')

In [49]:
merged_df = pd.merge(ndf, violation_defs_and_fines, on='Violation Code', how='left')

In [53]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8198207 entries, 0 to 8198206
Data columns (total 15 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Registration State     object        
 1   Plate Type             object        
 2   Violation Location     float64       
 3   Violation Precinct     int64         
 4   Issuer Precinct        int64         
 5   Issuer Code            int64         
 6   Violation Code         int64         
 7   Meter Number           object        
 8   Violation County       category      
 9   Issuer Squad           object        
 10  Issuing Agency         object        
 11  Issue Date and Time    datetime64[ns]
 12  Violation Description  object        
 13  Fine (below 96th)      float64       
 14  Fine                   float64       
dtypes: category(1), datetime64[ns](1), float64(3), int64(4), object(6)
memory usage: 946.0+ MB


In [50]:
fine_df = merged_df[['Violation Location', 'Violation County', 'Violation Description', 'Fine']].copy()

In [51]:
fine_df.head()

,Violation Location,Violation County,Violation Description,Fine
0,43.0,Bronx,PEDESTRIAN RAMP,165.0
1,14.0,Manhattan,FRAUDULENT USE PARKING PERMIT,65.0
2,14.0,NaN,NO STANDING-COMM METER ZONE,115.0
3,108.0,NaN,NO PARKING-DAY/TIME LIMITS,60.0
4,108.0,Queens,FIRE HYDRANT,115.0


In [69]:
time_df = ndf[['Issue Date and Time', 'Violation County', 'Violation Location', 'Violation Code']].copy()
time_df 

,Issue Date and Time,Violation County,Violation Location,Violation Code
0,2023-06-09 09:11:00,Bronx,43.0,67
1,2023-06-30 07:17:00,Manhattan,14.0,87
2,2023-06-30 08:23:00,NaN,14.0,31
3,2023-07-06 23:50:00,NaN,108.0,20
4,2023-07-08 23:50:00,Queens,108.0,40
...,...,...,...,...
8198202,2023-12-26 14:40:00,Brooklyn,67.0,14
8198203,2023-12-26 15:27:00,Brooklyn,71.0,19
8198204,2023-12-26 15:36:00,Brooklyn,71.0,71
8198205,2023-12-26 15:48:00,Brooklyn,67.0,20


In [72]:
time_df['Issue Date and Time'].describe(datetime_is_numeric=True).T

count      0
mean     NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: Issue Date and Time, dtype: object

In [70]:
mask = (time_df['Issue Date and Time'] >= '2016-07-01') & (time_df['Issue Date and Time'] < '2017-07-01')
time_df = time_df.loc[mask].copy()
time_df['Issue Date and Time'].describe(datetime_is_numeric=True)[['min', 'max']].T

min    NaT
max    NaT
Name: Issue Date and Time, dtype: object

In [71]:
index, county in enumerate(counties):
    county_df = df['Violation Location'][(df['Violation County'] == county) & 
                                         (df['Violation Location'].isin(precinct_dict[county])) &
                                         (df['Violation Location'].isin(feature_id_list))]

SyntaxError: invalid syntax (164043921.py, line 1)

In [68]:
time_df = time_df.set_index('Issue Date and Time', drop=True, append=False, inplace=False, verify_integrity=False).sort_index()

NameError: name 'time_df' is not defined

In [54]:
fine_precinct_df = fine_df.dropna(subset=['Violation Location', 'Violation County']).copy() 
#need to make this a string so the values can be compared witht the valied precincts 
fine_precinct_df['Violation Location'] = fine_precinct_df['Violation Location'].astype('int').astype('str')
# we select out only the valid precinct numbers 
fine_precinct_df = fine_precinct_df[fine_precinct_df['Violation Location'].isin(feature_id_list)] 
fine_precinct_df.sample(5)

,Violation Location,Violation County,Violation Description,Fine
2986521,6,Manhattan,NO STANDING-DAY/TIME LIMITS,115.0
6922690,14,Manhattan,PLTFRM LFTS LWRD POS COMM VEH,45.0
3370580,49,Bronx,FAIL TO DSPLY MUNI METER RECPT,35.0
3951552,67,Brooklyn,REG. STICKER-EXPIRED/MISSING,65.0
4172517,10,Manhattan,NO STANDING-DAY/TIME LIMITS,115.0


In [65]:
fine_precinct_df.to_csv('C:\\Users\\admin\\Downloads\\fine_precinct_df.csv')


In [ ]:
issuing_agency_df

In [55]:
import io

web_text = 'A-PORT AUTHORITY, B-TRIBOROUGH BRIDGE AND TUNNEL POLICE, C-CON RAIL, D-DEPARTMENT OF BUSINESS SERVICES, E-BOARD OF ESTIMATE, F-FIRE DEPARTMENT, G-TAXI AND LIMOUSINE COMMISSION, H-HOUSING AUTHORITY, I-STATEN ISLAND RAPID TRANSIT POLICE, J-AMTRAK RAILROAD POLICE, K-PARKS DEPARTMENT, L-LONG ISLAND RAILROAD, M-TRANSIT AUTHORITY, N-NYS PARKS POLICE, O-NYS COURT OFFICERS, P-POLICE DEPARTMENT, Q-DEPARTMENT OF CORRECTION, R-NYC TRANSIT AUTHORITY MANAGERS, S-DEPARTMENT OF SANITATION, T-TRAFFIC, U-PARKING CONTROL UNIT, V-DEPARTMENT OF TRANSPORTATION, W-HEALTH DEPARTMENT POLICE, X-OTHER/UNKNOWN AGENCIES, Y-HEALTH AND HOSPITAL CORP. POLICE, Z-METRO NORTH RAILROAD POLICE, 1-NYS OFFICE OF MENTAL HEALTH POLICE, 2-O M R D D, 3-ROOSEVELT ISLAND SECURITY, 4-SEA GATE ASSOCIATION POLICE, 5-SNUG HARBOR CULTURAL CENTER RANGERS, 6-WATERFRONT COMMISSION OF NY HARBOR, 7-SUNY MARITIME COLLEGE, 9-NYC OFFICE OF THE SHERIFF'
issuing_agency_df = pd.read_csv( io.StringIO(web_text), sep=',', header=None).T.rename(columns={0: 'Agency'})
issuing_agency_df = pd.DataFrame(issuing_agency_df.Agency.str.split('-',1).tolist(),
                                 columns = ['Issuing Agency','Agency Name'])
issuing_agency_df['Issuing Agency'] = issuing_agency_df['Issuing Agency'].str.strip() # Sees to still be whitespace so cnt merge properly
issuing_agency_df

C:\Users\admin\AppData\Local\Temp\ipykernel_25184\888015571.py:5: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  issuing_agency_df = pd.DataFrame(issuing_agency_df.Agency.str.split('-',1).tolist(),


,Issuing Agency,Agency Name
0,A,PORT AUTHORITY
1,B,TRIBOROUGH BRIDGE AND TUNNEL POLICE
2,C,CON RAIL
3,D,DEPARTMENT OF BUSINESS SERVICES
4,E,BOARD OF ESTIMATE
5,F,FIRE DEPARTMENT
6,G,TAXI AND LIMOUSINE COMMISSION
7,H,HOUSING AUTHORITY
8,I,STATEN ISLAND RAPID TRANSIT POLICE
9,J,AMTRAK RAILROAD POLICE


In [64]:
issuing_agency_df.to_csv('C:\\Users\\admin\\Downloads\\issuing_agency_df.csv')


In [ ]:
issuing_agency_df.to_csv('C:\\Users\\admin\\Downloads\\issuing_agency_df.csv')


In [56]:
merged_df2 = pd.merge(merged_df, issuing_agency_df, on='Issuing Agency', how='left')

In [57]:
merged_df2.head()

,Registration State,Plate Type,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Violation Code,Meter Number,Violation County,Issuer Squad,Issuing Agency,Issue Date and Time,Violation Description,Fine (below 96th),Fine,Agency Name
0,NY,PAS,43.0,43,43,972773,67,-,Bronx,0.0,P,2023-06-09 09:11:00,PEDESTRIAN RAMP,165.0,165.0,POLICE DEPARTMENT
1,NY,PAS,14.0,14,968,271057,87,-,Manhattan,0.0,M,2023-06-30 07:17:00,FRAUDULENT USE PARKING PERMIT,65.0,65.0,TRANSIT AUTHORITY
2,NY,PAS,14.0,14,968,271057,31,-,NaN,0.0,M,2023-06-30 08:23:00,NO STANDING-COMM METER ZONE,115.0,115.0,TRANSIT AUTHORITY
3,99,PAS,108.0,108,968,272834,20,-,NaN,0.0,M,2023-07-06 23:50:00,NO PARKING-DAY/TIME LIMITS,65.0,60.0,TRANSIT AUTHORITY
4,PA,PAS,108.0,108,968,272834,40,-,Queens,0.0,M,2023-07-08 23:50:00,FIRE HYDRANT,115.0,115.0,TRANSIT AUTHORITY


In [63]:
merged_df2.to_csv('C:\\Users\\admin\\Downloads\\merged_df2.csv')


In [58]:
temp = pd.DataFrame(merged_df2.groupby(['Agency Name', 'Violation County'])['Plate Type'].count().reset_index())
temp2 = temp.groupby(['Agency Name', 'Violation County'])['Plate Type'].sum().sort_values(ascending=False)
temp3 = pd.DataFrame(temp2).head(19)
temp4 = temp3.reset_index()
temp4

,Agency Name,Violation County,Plate Type
0,TRAFFIC,Manhattan,1577432
1,DEPARTMENT OF TRANSPORTATION,Queens,1495550
2,DEPARTMENT OF TRANSPORTATION,Brooklyn,1107524
3,TRAFFIC,Brooklyn,921463
4,TRAFFIC,Queens,880650
5,DEPARTMENT OF TRANSPORTATION,Bronx,554981
6,TRAFFIC,Bronx,440063
7,DEPARTMENT OF TRANSPORTATION,Manhattan,360428
8,DEPARTMENT OF TRANSPORTATION,Staten Island,272811
9,TRAFFIC,Staten Island,70586


In [61]:
temp.to_csv('C:\\Users\\admin\\Downloads\\temp.csv')


In [59]:
ndf.head()

,Registration State,Plate Type,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Violation Code,Meter Number,Violation County,Issuer Squad,Issuing Agency,Issue Date and Time
0,NY,PAS,43.0,43,43,972773,67,-,Bronx,0.0,P,2023-06-09 09:11:00
1,NY,PAS,14.0,14,968,271057,87,-,Manhattan,0.0,M,2023-06-30 07:17:00
2,NY,PAS,14.0,14,968,271057,31,-,NaN,0.0,M,2023-06-30 08:23:00
3,99,PAS,108.0,108,968,272834,20,-,NaN,0.0,M,2023-07-06 23:50:00
4,PA,PAS,108.0,108,968,272834,40,-,Queens,0.0,M,2023-07-08 23:50:00


In [60]:
ndf.to_csv('C:\\Users\\admin\\Downloads\\parking.csv')


In [62]:
ndf.head()

,Registration State,Plate Type,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Violation Code,Meter Number,Violation County,Issuer Squad,Issuing Agency,Issue Date and Time
0,NY,PAS,43.0,43,43,972773,67,-,Bronx,0.0,P,2023-06-09 09:11:00
1,NY,PAS,14.0,14,968,271057,87,-,Manhattan,0.0,M,2023-06-30 07:17:00
2,NY,PAS,14.0,14,968,271057,31,-,NaN,0.0,M,2023-06-30 08:23:00
3,99,PAS,108.0,108,968,272834,20,-,NaN,0.0,M,2023-07-06 23:50:00
4,PA,PAS,108.0,108,968,272834,40,-,Queens,0.0,M,2023-07-08 23:50:00
